#### Import libraries and setting up connection

In [5]:
from pymongo import MongoClient
import pandas as pd

In [6]:
client = MongoClient()

In [7]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [8]:
db = client.get_database("companies")

In [9]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

### 1. Designers talk and like to share knowledge

In [10]:
s = print(list(db.companies.distinct("category_code")))

[None, 'advertising', 'analytics', 'automotive', 'biotech', 'cleantech', 'consulting', 'design', 'ecommerce', 'education', 'enterprise', 'fashion', 'finance', 'games_video', 'government', 'hardware', 'health', 'hospitality', 'legal', 'local', 'manufacturing', 'medical', 'messaging', 'mobile', 'music', 'nanotech', 'network_hosting', 'news', 'nonprofit', 'other', 'photo_video', 'public_relations', 'real_estate', 'search', 'security', 'semiconductor', 'social', 'software', 'sports', 'transportation', 'travel', 'web']


In [11]:
#set([db.companies['category_code']])

#### I want to see where the design companies are located, therefore I look for them in mongo db

In [12]:
dc = list(db.companies.find({"category_code":'design'},
                       {'offices.country_code':1}))

In [13]:
dc

[{'_id': ObjectId('52cdef7c4bab8bd675298447'),
  'offices': [{'country_code': 'USA'},
   {'country_code': 'AUS'},
   {'country_code': 'DEU'}]},
 {'_id': ObjectId('52cdef7e4bab8bd67529ba4e'),
  'offices': [{'country_code': 'USA'}]},
 {'_id': ObjectId('52cdef7e4bab8bd67529bd1a'),
  'offices': [{'country_code': 'USA'}]},
 {'_id': ObjectId('52cdef7f4bab8bd67529c47a'),
  'offices': [{'country_code': 'GBR'}]}]

#### As we can see out of the 6 design offices in the company list, 3 of them are located in the USA, to check the data more into detail I want to see not only the country but the city and state

In [14]:
dccs = list(db.companies.find({'category_code':'design'},
                       {'offices.city':1,'offices.state_code':1,'offices.country_code':1}))

In [15]:
dccs

[{'_id': ObjectId('52cdef7c4bab8bd675298447'),
  'offices': [{'city': 'San Francisco',
    'state_code': 'CA',
    'country_code': 'USA'},
   {'city': 'Collingwood', 'state_code': None, 'country_code': 'AUS'},
   {'city': 'Berlin', 'state_code': None, 'country_code': 'DEU'}]},
 {'_id': ObjectId('52cdef7e4bab8bd67529ba4e'),
  'offices': [{'city': 'Ellensburg',
    'state_code': 'WA',
    'country_code': 'USA'}]},
 {'_id': ObjectId('52cdef7e4bab8bd67529bd1a'),
  'offices': [{'city': 'Brooklyn',
    'state_code': 'NY',
    'country_code': 'USA'}]},
 {'_id': ObjectId('52cdef7f4bab8bd67529c47a'),
  'offices': [{'city': 'London', 'state_code': None, 'country_code': 'GBR'}]}]

#### I also have to take into consideration that offices can have more than 1 object, therefor I need to use unwind

In [16]:
res = db.companies.aggregate([
                      {"$unwind":"$offices" },
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$project":{"_id":0}} # We omit the id, because when inserting into new collection, we can't
                                             # have two documents with same id.
                     ])

In [17]:
res

In [18]:
db.companies_unwinded.insert_many(res)

In [19]:
offices = db.companies_unwinded

In [20]:
res = offices.find({},{"offices":1})

In [21]:
#therefor the remaining results for: NO null in office on lat and long and design as category code

In [22]:
desf = list(db.companies_unwinded.find({'category_code':'design'},
                       {'offices.city':1,'offices.state_code':1,'offices.country_code':1}))

In [23]:
desf

[{'_id': ObjectId('5fb2ca3cf3a429eb4e3fd1c8'),
  'offices': {'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA'}},
 {'_id': ObjectId('5fb2ca3cf3a429eb4e3fd1c9'),
  'offices': {'city': 'Collingwood',
   'state_code': None,
   'country_code': 'AUS'}},
 {'_id': ObjectId('5fb2ca3cf3a429eb4e3fd1ca'),
  'offices': {'city': 'Berlin', 'state_code': None, 'country_code': 'DEU'}},
 {'_id': ObjectId('5fb2ca3df3a429eb4e3fefdd'),
  'offices': {'city': 'Ellensburg',
   'state_code': 'WA',
   'country_code': 'USA'}}]

#### From this first filter, we obtain 4 companies from the design sector in the following cities:
- San Francisco
- Collingwood
- Berlin
- Ellensburg

## 2. Lets investigate a little about tech startups

In [24]:
#from the list of category code, we suppose that all of them that contain the term tech in it are from that sector

In [25]:
s = print(list(db.companies.distinct("category_code")))

[None, 'advertising', 'analytics', 'automotive', 'biotech', 'cleantech', 'consulting', 'design', 'ecommerce', 'education', 'enterprise', 'fashion', 'finance', 'games_video', 'government', 'hardware', 'health', 'hospitality', 'legal', 'local', 'manufacturing', 'medical', 'messaging', 'mobile', 'music', 'nanotech', 'network_hosting', 'news', 'nonprofit', 'other', 'photo_video', 'public_relations', 'real_estate', 'search', 'security', 'semiconductor', 'social', 'software', 'sports', 'transportation', 'travel', 'web']


In [26]:
filter_01 = {'category_code':{'$regex':'tech$'},'total_money_raised':{'$gte':'$1_000_000'}}

In [27]:
filter_1 = list(db.companies.find(filter_01))

In [28]:
len(filter_1)

361

In [29]:
#tenindo en cuenta los unwinded el len se reduce

In [30]:
filter_01 = {'category_code':{'$regex':'tech$'},'total_money_raised':{'$gte':'$1_000_000'}}

In [31]:
filter_1 = list(db.companies_unwinded.find(filter_01))

In [32]:
len(filter_1)

263

#### This means there is about 263 companies that fulfill that requirement.


### Now, we combine both filters (design and tech) and see from those 263 tech companies, how many are also close to a design company (from the 4 obtained above)

#### We start with SanFrancisco

In [33]:
filter_tandc1 = {'category_code':{'$regex':'tech$'},
                 'total_money_raised':{'$gte':'$1_000_000'},
                 'offices.city':'San Francisco'}

In [34]:
filter_sf = list(db.companies_unwinded.find(filter_tandc1))

In [35]:
len(filter_sf)

5

#### The we check Collingwood

In [36]:
filter_tandc2 = {'category_code':{'$regex':'tech$'},'total_money_raised':{'$gte':'$1_000_000'},'offices.city':'Collingwood'}

In [37]:
filter_c= list(db.companies_unwinded.find(filter_tandc2))

In [38]:
len(filter_c)

0

#### FAollowing up we check Berlin

In [39]:
filter_tandc3 = {'category_code':{'$regex':'tech$'},'total_money_raised':{'$gte':'$1_000_000'},'offices.city':'Berlin'}

In [40]:
filter_b= list(db.companies_unwinded.find(filter_tandc3))

In [41]:
len(filter_b)

0

#### And at last Ellensburg

In [42]:
filter_tandc4 = {'category_code':{'$regex':'tech$'},'total_money_raised':{'$gte':'$1_000_000'},'offices.city':'Ellensburg'}

In [43]:
filter_e= list(db.companies_unwinded.find(filter_tandc4))

In [44]:
len(filter_e)

0

#### From this quick analysis we can see how SanFrancisco is the only city with tech companies

## To finnish with these to first filters, I check how many companies there are close to what could be the chosen city

## Geoqueries

In [45]:
#In geoqueries, the coordinates are inversed, hence why I do the following

In [46]:
res = offices.find({},{"offices":1})

In [47]:
for c in res:
    filt = {"_id":c["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [c["offices"]["longitude"], c["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices.update_one(filt,update)

### Near a point

In [48]:
# I will be using the coordinates of all 4 of the result obtaines above, in order to check wich 
# one obtains more results, and therefore is the most suitable 

In [49]:
desff = list(db.companies_unwinded.find({'category_code':'design'},
                       {'offices.city':1,'offices.country_code':1,'offices.latitude':1, 'offices.longitude':1}))

In [50]:
desff

[{'_id': ObjectId('5fb2ca3cf3a429eb4e3fd1c8'),
  'offices': {'city': 'San Francisco',
   'country_code': 'USA',
   'latitude': 37.7955307,
   'longitude': -122.4005983}},
 {'_id': ObjectId('5fb2ca3cf3a429eb4e3fd1c9'),
  'offices': {'city': 'Collingwood',
   'country_code': 'AUS',
   'latitude': -37.8026587,
   'longitude': 144.9868546}},
 {'_id': ObjectId('5fb2ca3cf3a429eb4e3fd1ca'),
  'offices': {'city': 'Berlin',
   'country_code': 'DEU',
   'latitude': 52.4986199,
   'longitude': 13.4469031}},
 {'_id': ObjectId('5fb2ca3df3a429eb4e3fefdd'),
  'offices': {'city': 'Ellensburg',
   'country_code': 'USA',
   'latitude': 45.7968414,
   'longitude': -122.6931439}}]

In [51]:
#offices.create_index([('coord','2dsphere')])

### SF office

In [52]:
SF_point = {"type":"Point",
         "coordinates":[-122.4005983,37.7955307]}

query ={
    "coord":{
        "$near":{
            "$geometry":SF_point,
            "$maxDistance":1_000,#In meters
            #"$minDistance":
        }
    }
}


res_SF = db.companies_unwinded.find(query,{"name":1})

In [53]:
len(list(res_SF))

214

### Collingwood office

In [54]:
Cll_point = {"type":"Point",
         "coordinates":[144.9868546,-37.8026587]}

query ={
    "coord":{
        "$near":{
            "$geometry":Cll_point,
            "$maxDistance":1_000,#In meters
            #"$minDistance":
        }
    }
}


res_Cll = db.companies_unwinded.find(query,{"name":1})

In [55]:
len(list(res_Cll))

2

### Berlin office

In [56]:
B_point = {"type":"Point",
         "coordinates":[13.4469031,52.4986199]}

query ={
    "coord":{
        "$near":{
            "$geometry":B_point,
            "$maxDistance":1_000,#In meters
            #"$minDistance":
        }
    }
}


res_B =db.companies_unwinded.find(query,{"name":1})

In [57]:
len(list(res_B))

6

### Ellensburg office

In [58]:
E_point = {"type":"Point",
         "coordinates":[-122.6931439,45.7968414]}

query ={
    "coord":{
        "$near":{
            "$geometry":E_point,
            "$maxDistance":1_000,#In meters
            #"$minDistance":
        }
    }
}


res_E = db.companies_unwinded.find(query,{"name":1})

In [59]:
len(list(res_E))

2

### From this analysis, we corroborate that San Francisco is the chosen city, beacuse its the only one that fullfills both the requirements set above, and is also the city with the most companies 

## 3. Account managers need to travel a lot, therefore I choose the Airport as my 3rd filter

In [60]:
#SF international airpot == 'lat': 37.6167130000997,'lng': -122.38709449768066

In [61]:
travel_point = {"type":"Point",
         "coordinates":[-122.38709449768066,37.6167130000997]}

query ={
    "coord":{
        "$near":{
            "$geometry":travel_point,
            "$maxDistance":3_300,#In meters
            #"$minDistance":
        }
    }
}


res_travel = db.companies_unwinded.find(query,{"name":1,'offices.country_code':1,'offices.latitude':1, 'offices.longitude':1})

In [62]:
travel = list(res_travel)

In [63]:
travel

[{'_id': ObjectId('5fb2ca3df3a429eb4e3ff1ed'),
  'name': 'Gen-Y Media',
  'offices': {'country_code': 'USA',
   'latitude': 37.598478,
   'longitude': -122.376572}},
 {'_id': ObjectId('5fb2ca3df3a429eb4e3ff4e7'),
  'name': 'Sky Bird Travel  Tours',
  'offices': {'country_code': 'USA',
   'latitude': 37.599131,
   'longitude': -122.368281}},
 {'_id': ObjectId('5fb2ca3cf3a429eb4e3fd2c5'),
  'name': 'ScanCafe',
  'offices': {'country_code': 'USA',
   'latitude': 37.592584,
   'longitude': -122.369676}},
 {'_id': ObjectId('5fb2ca3cf3a429eb4e3fe3e2'),
  'name': 'Planet Metrics',
  'offices': {'country_code': 'USA',
   'latitude': 37.628328,
   'longitude': -122.420814}},
 {'_id': ObjectId('5fb2ca3df3a429eb4e3ff382'),
  'name': 'Catalyst Biosciences',
  'offices': {'country_code': 'USA',
   'latitude': 37.645401,
   'longitude': -122.394464}}]

#### From this third filter I obtain 5 different companies that follow these filters;
- Close to a design company
- Close to a tech company
- Close to the airport

In [64]:
%store travel

Stored 'travel' (list)
